# Llama 3.1 Hugging Face Hub - ModelBuilder

This notebook was tested with the `conda_python3` kernel on an Amazon SageMaker notebook instance of type `g5.2xl` with a 100GB EBS volume attached. If you are not using local mode, feel free to make use of a smaller CPU instance type / EBS volume. 

In [1]:
# !pip install boto3 sagemaker -U --quiet

In [2]:
! pip list | grep sagemaker

sagemaker                     2.228.0
sagemaker_pyspark             1.4.5


# SageMaker Model Builder experience

In this example, we are using ModelBuilder to deploy an Llama 2 model directly. You can use `Mode` to switch between local testing and deploying to a SageMaker Endpoint. 

### SageMaker ModelBuilder: Local deployment

Now we will use SageMaker ModelBuilder class to prepare the model for local and remote deployment.

In [3]:
from sagemaker import get_execution_role, Session, image_uris
import boto3
import os
sagemaker_session = Session()
region = boto3.Session().region_name

# get execution role
# please use execution role if you are using notebook instance or update the role arn if you are using a different role
execution_role = get_execution_role() if get_execution_role() is not None else "your-role-arn"

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/SageMaker/.xdg/config/sagemaker/config.yaml


In [4]:
path = os.path.join(os.getcwd(), 'Llama-3.1-8B-Instruct') 
os.makedirs(path, exist_ok=True)
print(path)

/home/ec2-user/SageMaker/sagemaker-hosting/SageMaker-Model-Builder/foundation-models/Llama-3.1-8B-Instruct


[Llama 2 is a gated model](https://huggingface.co/meta-llama/Llama-2-7b-hf) and requires access to be approved. Once approved you will need to pass your [Hugging Face access token](https://huggingface.co/docs/hub/security-tokens) as seen in the below cell. 

In [5]:
import os
from getpass import getpass
 
is_sagemaker_notebook = True
# is_sagemaker_notebook = False # use VS Code

if is_sagemaker_notebook:
    HF_TOKEN = getpass("Enter HUGGINGFACE Access Token: ")
else: # VS Code
    from dotenv import load_dotenv
    HF_TOKEN = os.getenv('HF_TOKEN') or getpass("Enter HUGGINGFACE Access Token: ")
    print("token: ", HF_TOKEN)

In [6]:
from sagemaker.serve.builder.model_builder import ModelBuilder
from sagemaker.serve.builder.schema_builder import SchemaBuilder
from sagemaker.serve import Mode
import json

prompt = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the"
response = "The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the east coast."

sample_input = {
    "inputs": prompt,
    "parameters": {}
}

sample_output = [
    {
        "generated_text": response
    }
]

model_builder = ModelBuilder(
    # model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    model = "meta-llama/Meta-Llama-3-8B",
    schema_builder=SchemaBuilder(sample_input, sample_output),
    model_path=path, #local path where artifacts will be saved
    mode=Mode.LOCAL_CONTAINER, # The model will be deployed locally. Change to Mode.SAGEMAKER_ENDPOINT to deploy to a SageMaker endpoint. 
    env_vars={
        "HUGGING_FACE_HUB_TOKEN": HF_TOKEN # Llama 2 is a gated model and requires a Hugging Face Hub token. 
    }
)

In [7]:
model = model_builder.build()

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
ModelBuilder: INFO:     Either inference spec or model is provided. ModelBuilder is not handling MLflow model input
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: WARNING:     HuggingFace JumpStart Model ID not detected. Building for HuggingFace Model ID.
ModelBuilder: WARNING:     HuggingFace Model ID support on model server: None is not currently supported. Defaulting to TGI
ModelBuilder: INFO:     Local instance_type ml.g5.12xlarge detected. ml.g5.12xlarge will be default when deploying to a SageMaker Endpoint. This default can be overriden in model.deploy()
ModelBuilder: WARNING:     91.85528052148224 percent of docker disk space at /var/lib/docker is used. Please consider freeing up disk space or increasing the EBS volume if you are on a SageMaker Notebook.
ModelBuilder: INFO:     CUDA enabled hardware on the devic

### Tune the model container locally to get the best configuration for deployment

A neat feature of Model Builder is the ability to run local tuning of the container parameter(s) when you use the LOCAL_CONTAINER mode.

In [8]:
%%time
tuned_model = model.tune()

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
ModelBuilder: INFO:     CUDA enabled hardware on the device: ['NVIDIA A10G, 22723 MiB', 'NVIDIA A10G, 22723 MiB', 'NVIDIA A10G, 22723 MiB', 'NVIDIA A10G, 22723 MiB']
ModelBuilder: INFO:     Model can be sharded across [4, 2, 1] GPUs
ModelBuilder: INFO:     Trying num shard: 4, dtype: bfloat16...
ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using

In [15]:
local_predictor = tuned_model.deploy()

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
ModelBuilder: INFO:     Pulling image 763104351884.dkr.ecr.us-east-1.amazonaws.com/huggingface-pytorch-tgi-inference:2.3.0-tgi2.2.0-gpu-py310-cu121-ubuntu22.04-v2.0 from repository...
ModelBuilder: DEBUG:     Stopping currently running container...
ModelBuilder: INFO:     Waiting for model server TGI to start up...
ModelBuilder: DEBUG:     2024-08-11T03:10:11.559651Z  INFO text_generation_launcher: Args {
ModelBuilder: DEBUG:         model_id: "meta-llama/Llama-2-7b-hf",
ModelBuilder: DEBUG:         revision: None,
ModelBuilder: DEBUG:         validation_workers: 2,
ModelBuilder: DEBUG:         sharded: Some(
Mode

In [16]:
updated_sample_input = model_builder.schema_builder.sample_input
updated_sample_input

{'inputs': 'The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the',
 'parameters': {'max_new_tokens': 128}}

In [17]:
%%time
print(local_predictor.predict(updated_sample_input))

[{'generated_text': ' eastern United States. The diamondback terrapin is the only species in the genus Malaclemys.\nThe diamondback terrapin is a medium-sized turtle, with a carapace length of 10–15 cm (4–6 in). The shell is oval in shape, with a high, domed carapace and a flattened plastron. The carapace is dark brown to black, with a yellowish-brown to orange-brown stripe running down the middle of each scute. The plastron is yellowish-'}]
CPU times: user 4.08 ms, sys: 0 ns, total: 4.08 ms
Wall time: 1.44 s


### SageMaker ModelBuilder: Deploy to a SageMaker Endpoint

Now we have tested the model prediction locally, we can continue to deploy the model to a SageMaker endpoint.

In [19]:
predictor = tuned_model.deploy(mode=Mode.SAGEMAKER_ENDPOINT, role=execution_role)

ModelBuilder: INFO:     ModelBuilder will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features. To opt out of telemetry, please disable via TelemetryOptOut in intelligent defaults. See https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk for more info.
ModelBuilder: WARNING:     Deploying in SAGEMAKER_ENDPOINT Mode, overriding existing configurations set for LOCAL_CONTAINER mode
ModelBuilder: DEBUG:     Uploading TGI Model Resources uncompressed to: s3://sagemaker-us-east-1-057716757052/huggingface-pytorch-tgi-inference-2024-08-11-03-10-46-380/code
INFO:sagemaker:Creating model with name: huggingface-pytorch-tgi-inference-2024-08-11-03-15-02-850
INFO:sagemaker:Creating endpoint-config with name huggingface-pytorch-tgi-inference-2024-08-11-03-15-03-574
INFO:sagemaker:Creating endpoint with name huggingface-pytorch-tgi-inference-2024-08-11-03-15-03-574


-----------!

ModelBuilder: DEBUG:     ModelBuilder metrics emitted.


In [23]:
updated_sample_input

{'inputs': 'The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the',
 'parameters': {'max_new_tokens': 128}}

In [20]:
%%time
print(predictor.predict(updated_sample_input)[0]["generated_text"])

The diamondback terrapin or simply terrapin is a species of turtle native to the brackish coastal tidal marshes of the eastern United States. The diamondback terrapin is the only species in the genus Malaclemys.
The diamondback terrapin is a medium-sized turtle, with a carapace length of 10–15 cm (4–6 in). The shell is oval in shape, with a high, rounded keel running down the middle of the back. The shell is dark brown to black, with a yellowish-orange stripe running down the middle of the back. The head is small and triangular in shape, with a pointed snout. The eyes are
CPU times: user 11.6 ms, sys: 1.66 ms, total: 13.2 ms
Wall time: 1.54 s


## Clean up

In [22]:
local_predictor.delete_predictor()
predictor.delete_model()
predictor.delete_endpoint()

INFO:sagemaker:Deleting model with name: huggingface-pytorch-tgi-inference-2024-08-11-03-15-02-850
INFO:sagemaker:Deleting endpoint configuration with name: huggingface-pytorch-tgi-inference-2024-08-11-03-15-03-574
INFO:sagemaker:Deleting endpoint with name: huggingface-pytorch-tgi-inference-2024-08-11-03-15-03-574
